In [34]:
import math, json, redshift

## Extinction, galactic latitude and peak absolute magnitude
This notebook computes three Lasair features: extinction E(B-V), galactic latitude, 
and peak extinction-corrected absolute magnitude (PECAM).

### 1. Extinction
#### Get the dustmaps package set up. 
We use this package to compute the extinction from the dustmap of 
[Schlegel, Finkbeiner, and Davis](https://iopscience.iop.org/article/10.1086/305772).
Once the package is installed, the dustmaps themselves must be downloaded with fetch() -- change the
directory name according to your own setup.
Note that the fetch() method only needs to be called once. 

In [35]:
#!/usr/bin/pip3 install dustmaps
from dustmaps.config import config
config['data_dir'] = '/Users/rwillia5/Library/Python/3.9/lib/python/site-packages/dustmaps/data/sfd/'
import dustmaps.sfd
#dustmaps.sfd.fetch()
from dustmaps.sfd import SFDQuery
from astropy.coordinates import SkyCoord
sfd = SFDQuery()

#### Corrected colour for extinction
These colour corrections are multiplied by the extinction E(B-V) to get a magnitude correction. 
They arefrom Table 6 of 
[Schlafly and Finkbeiner](https://iopscience.iop.org/article/10.1088/0004-637X/737/2/103) with RV=3.1

In [36]:
# The LSST bands
EXTCOEF   = {'u':4.145, 'g':3.237, 'r':2.273, 'i':1.684, 'z':1.323, 'y':1.088}

# Modify magnitude for extinction
def dustmag(mag, band, ebv):
    return mag - ebv*EXTCOEF[band]

#### Extinction from sky position

In [37]:
def computeExtinction(ra, decl):
    c = SkyCoord(ra, decl, unit="deg", frame='icrs')
    ebv = float(sfd(c))
    return ebv

### 2. Magnitude and Flux
#### Transform between magnitudes and nanoJanskies

In [38]:
def mag2flux(mag):
    # flux in nanoJ
    flux =  math.pow(10, (31.4 - mag)/2.5)
    return flux
def flux2mag(flux):
    # flux in nanoJ
    mag = 31.4 - 2.5*math.log10(flux)
    return mag

### 3. Peak extinction-corrected magnitude
From the E(B-V) and lightcurve in flux, we compute magnitude, 
correct them with extinction, then find the brightest point, and the associated band.

In [39]:
def findPeakExtMag(ebv, lightcurve):
    peakMag  = 100
    peakBand = None
    for diaSource in lightcurve:
        mjd        = diaSource['midpointMjdTai']
        band       = diaSource['band']
        psfFlux    = diaSource['psfFlux']
        psfFluxErr = diaSource['psfFluxErr']
        if psfFlux > 0:
            mag = flux2mag(psfFlux)
            correctedMag = mag - ebv*EXTCOEF[band]
            if correctedMag < peakMag:
                peakMag = correctedMag
                peakBand = band
    return (peakMag, peakBand)

### 4. Galactic latitude

In [40]:
# https://en.wikipedia.org/wiki/Galactic_coordinate_system
def galacticLat(ra, decl):
    alphaNGP = 192.85948
    deltaNGP =  27.1283
    sdngp = math.sin(math.radians(deltaNGP))
    cdngp = math.cos(math.radians(deltaNGP))
    sde = math.sin(math.radians(decl))
    cde = math.cos(math.radians(decl))
    cra = math.cos(math.radians(ra - alphaNGP))
    glat = math.degrees(math.asin(sdngp*sde + cdngp*cde*cra))
    return glat

### 5. Test run
Use the Lasair API to find some objects with a host galaxy, then compute their features.
Make sure to connect to the right endpoint.
The `cache` argument is the name of a directory where API calls can be cached, so you don't
need to go to the Lasair server for the same query a second time (can be omotted).

In [41]:
#!/usr/bin/pip3 install lasair
from lasair import LasairError, lasair_client as lasair
import settings
endpoint = "https://lasair-lsst-dev.lsst.ac.uk/api"
L = lasair(settings.API_TOKEN, endpoint=endpoint, cache='cache')

#### Get some working objects with a Sherlock redshift

In [42]:
selected = """ 
  objects.diaObjectId, objects.ra, objects.decl, 
  sherlock_classifications.z, sherlock_classifications.photoz
"""
tables = 'objects,sherlock_classifications'
conditions = """
  classification="SN" AND 
  (sherlock_classifications.z > 0 OR sherlock_classifications.photoz > 0)
"""
results = L.query(selected, tables, conditions, limit = 50)
for row in results:
    objectId = row['diaObjectId']
    ra       = row['ra']
    decl     = row['decl']
    z        = row['z']
    photoz   = row['photoz']
    if not z:
        z = photoz
    object   = L.object(objectId, lasair_added=False, lite=True)

    # compute galactic latitude
    ebv = computeExtinction(ra, decl)
    
    # compute extinction
    galLat = galacticLat(ra, decl)

    # compute peak extinction-corrected apparent magnitude
    (peakMag, peakBand) = findPeakExtMag(ebv, object['diaSources'])
    if not peakBand:
        continue

    # combine z and apparent mag to get absolute mag
    distances = redshift.redshiftToDistance(z)
    distanceModulus = distances['dmod']

    absMag = peakMag - distanceModulus

    print('%s: ra=%.2f decl=%.2f z=%.2f E(B-V)=%.2f peakMax=%.2f peakBand=%s, absMag=%.2f' % 
          (objectId, ra, decl, z, ebv, peakMag, peakBand, absMag))


3516505582238433299: ra=55.08 decl=-26.84 z=0.01 E(B-V)=0.01 peakMax=19.16 peakBand=u, absMag=-12.90
3516505585459658821: ra=54.87 decl=-27.29 z=0.10 E(B-V)=0.01 peakMax=22.00 peakBand=z, absMag=-16.43
3516505593512722439: ra=52.41 decl=-27.25 z=0.24 E(B-V)=0.01 peakMax=22.75 peakBand=i, absMag=-17.61
3516505593512722444: ra=52.43 decl=-27.29 z=0.04 E(B-V)=0.01 peakMax=23.01 peakBand=u, absMag=-13.05
3516505600492044306: ra=53.64 decl=-27.28 z=0.00 E(B-V)=0.01 peakMax=19.29 peakBand=u, absMag=-11.89
3516505600492044307: ra=53.64 decl=-27.29 z=0.00 E(B-V)=0.01 peakMax=19.36 peakBand=i, absMag=-11.81
3516505600492044308: ra=53.57 decl=-27.29 z=0.00 E(B-V)=0.01 peakMax=19.65 peakBand=y, absMag=-11.52
3516505600492044311: ra=53.59 decl=-27.31 z=0.00 E(B-V)=0.01 peakMax=20.18 peakBand=g, absMag=-11.00
3516505600492044313: ra=53.59 decl=-27.33 z=0.00 E(B-V)=0.01 peakMax=20.46 peakBand=u, absMag=-10.72
3516505600492044348: ra=53.62 decl=-27.33 z=0.00 E(B-V)=0.01 peakMax=21.52 peakBand=y, absM